# Here we will try to adapt our environment to OPENAI GYM

In [1]:
pip install gym


     -------------------------------------- 721.7/721.7 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827647 sha256=cea587b6cb4fea7d7cef1bf6b4efdb3c0ec4a1bd8dc76d88d50e4e7eecd075d8
  Stored in directory: c:\users\felip\appdata\local\pip\cache\wheels\ae\5f\67\64914473eb34e9ba89dbc7eefe7e9be8f6673fbc6f0273b29f
Successfully built gym
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\felip\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import gym
from gym import spaces
import random
import copy as cp
import itertools

### Previous Implementation

In [4]:
class Card:
    def __init__(self, level, color, cost, points):
        '''Cards are defined by level color cost and points they give'''
        #int ranges from 1 to 3
        self.level = level
        #We would keep as int based on this mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}(alphabetical order)
        self.color = color
        #We do cost as an array [black, ...., white] [0, 2, 1, 0, 0]
        self.cost = cost
        #int, ranges from 0 to 5
        self.points = points

class Noble:
    def __init__(self, requirements, points):
        '''For nobles we only care about requirements and points you get'''
        #We do requirements as an array [black, ...., white] [3, 3, 3, 0, 0]
        self.requirements = requirements
        #int ranges from 3 to 4
        self.points = points

class GemColor:
    def __init__(self, name):
        self.name = name

In [5]:
class SplendorAgent:
    '''Our agent will, based on its state(gems/cards/nobles) get the set of legal actions and then execute the chosen one'''
    
    def __init__(self, gem_reserve=None, cards_player=None, players_state=None, nobles=None):
        self.gem_reserve = gem_reserve if gem_reserve is not None else [0,0,0,0,0,0] # gems in the agents hand initializes as 0
        self.player_cards = cards_player if cards_player is not None else [] # cars in the player power
        self.player_reserved_cards = [] #cards reserved by the player
        self.nobles = nobles if nobles is not None else [] #nobles the player has
        self.buying_power = self.gem_reserve # buying power is initializes equal to the amount of gems of each kind
        self.points = 0

        self.binary_lists_with_sum_of_3 = list(filter(lambda binary_list: sum(binary_list) == 3, itertools.product([0, 1], repeat=5)))
        self.binary_lists_with_sum_of_2 = list(filter(lambda binary_list: sum(binary_list) == 2, itertools.product([0, 1], repeat=5)))
    
    def reset(self):
        self.gem_reserve = [0, 0, 0, 0, 0, 0] # Reset gems in the agent's hand to 0
        self.player_cards = [] # Reset cards in the player's possession
        self.player_reserved_cards = [] # Reset reserved cards
        self.nobles = [] # Reset nobles
        self.buying_power = [0, 0, 0, 0, 0, 0] # Reset buying power to 0
        self.points = 0 # Reset points to 0

        self.binary_lists_with_sum_of_3 = list(filter(lambda binary_list: sum(binary_list) == 3, itertools.product([0, 1], repeat=5)))
        self.binary_lists_with_sum_of_2 = list(filter(lambda binary_list: sum(binary_list) == 2, itertools.product([0, 1], repeat=5)))

    def UpdateBuyingPower(self): 
        ''' Buying power is gems + cards bought of each color'''
        self.buying_power =cp.deepcopy(self.gem_reserve)
        for card in self.player_cards:
            self.buying_power[card.color] +=1 # if you have bought a card of said color you get an extra buying unit for that color.

    def get_legal_actions(self, spl_env):
        '''Function to get available actions for the agent based on the state'''
        MAX_GEMS_SUPPLY=10 #general attribute to call and compare againtst gems
        MAX_JOKERS=3

        print('Gem_supply:',spl_env.gems)
        legal_actions = []
        '''We check what cards can be bought based on the updated buying power taking into account any jokers in hand'''
        #can buy card #TO BE PUT IN A NICE LOOKING FUNCTION
        card_buy =[]
        for board in [spl_env.board_1, spl_env.board_2, spl_env.board_3]:
                
                for card in board:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])
                    if over <= self.buying_power[-1]:
                        card_buy.append(card)

        card_buy = [('buy card', x) for x in card_buy]
        legal_actions.extend(card_buy)

        '''We check what cards can be reserved'''

        #RESERVE #TO BE PUT IN A NICE LOOKING FUNCTION
        reserve_cards = []
        if spl_env.gems[-1] >0 and len(self.player_reserved_cards) <MAX_JOKERS:
            for board in [spl_env.board_1, spl_env.board_2, spl_env.board_3]:
                    for card in board:
                            reserve_cards.append(card)
        reserve_cards = [('reserve', x) for x in reserve_cards]
        legal_actions.extend(reserve_cards)

        '''We check what reserved cards can be bought based on the updated buying power taking into account any jokers in hand'''
        #Buy reserved cards
        buyable_reserved_cards = []
     
        for card in self.player_reserved_cards:
                    over = 0
                    for i in range(len(card.cost)):
                        over += max(0, card.cost[i]- self.buying_power[i])

                    if over <= self.buying_power[-1]:
                        buyable_reserved_cards.append(card)
        buyable_reserved_cards = [('buy reserved', x) for x in buyable_reserved_cards]
        legal_actions.extend(buyable_reserved_cards)

        #Take Gems
        '''We check all possible combinations of gem taking doubles and triples.'''

        #make the list of gems currently takable
        doable_3gems = []
        for comb in self.binary_lists_with_sum_of_3:
            if  min([spl_env.gems[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_3gems.append(comb)

        doable_2gems = []
        for comb in self.binary_lists_with_sum_of_2:
            if  min([spl_env.gems[i] - comb[i] for i in range(len(comb))]) >=0:
                doable_2gems.append(comb)
            
        
        gem_buy= []

        # If gem supply is 10, don't consider taking gems
        if sum(self.gem_reserve) == MAX_GEMS_SUPPLY:
             return legal_actions
             
        #append the list of 3 gems if the player has already 7 gems or less
        if sum(self.gem_reserve) <= 7:
            gem_buy.extend(doable_3gems)

        #append the list of 2 gems if the player has already 8

        if sum(self.gem_reserve) == 8:
            gem_buy.extend(doable_2gems)
        
        # append the list of 1 gem if the player has already 9
        
        if sum(self.gem_reserve) == 9:
            gem_buy.extend([[1,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]])

        #Appends the duos
        ##If the supply of gems is greater than 3 we can alwasy take 2 of the same gems.
        for x in range(len(spl_env.gems[:5])):
                list_empty = [0,0,0,0,0]
                if spl_env.gems[x] >3:
                    list_empty[x] = 2
                    gem_buy.append(list_empty)
        gem_buy = [('take', x) for x in gem_buy]
        legal_actions.extend(gem_buy)
        return legal_actions

        ###There is a bug here in the legal actions, not getting all posibilities need to fix!!


    def execute_action(self, action, spl_env): 
        '''Once an actions has been chosen we execute it by calling the action function'''

        # execute the specified action and update the state
        action_type, action_params = action
        if action_type == 'buy card':
            card = action_params
            self.buy_card(card, spl_env)

        elif action_type == 'buy reserved':
            card = action_params
            self.buy_reserved_card(card, spl_env)

        elif action_type == 'reserve':
            card = action_params
            self.reserve_card(card, spl_env)

        elif action_type == 'take':
            noble = action_params
            self.take_gems(noble, spl_env)

    def buy_card(self, card, spl_env):
        '''We buy the selected card and update the environment by removing the bought card from the board, updating player's gems and supply gems'''
        # update gem and card supply and player's state when a card is bought
        #Before adding card to players hand we need to check how much buying power do our current cards give us:
        #extract the colors of the current cards
        cards_color = [x.color for x in self.player_cards]

        #get our current color portofolio = buying power without gems
        cards_sum = [0,0,0,0,0]
        for x in cards_color:
             cards_sum[x] +=1
        
        #add card to players cards
        self.player_cards.append(card)

        ##Now we remove it from the board
        print('cards power:',cards_sum)
        print('cost:',card.cost)
        if card.level ==1:
             spl_env.board_1.remove(card)
        
        elif card.level ==2:
             spl_env.board_2.remove(card)

        elif card.level ==1:
             spl_env.board_3.remove(card)

        ##First check from the cost we substract our current free buying power.
        new_cost=[0,0,0,0,0]
        for i in range(len(card.cost)):
            new_cost[i]=max(0,card.cost[i]-cards_sum[i])
        
        print('Discounted card cost:',new_cost)


        #compute required tokens
        over = 0
        print('orig gem reserve:',self.gem_reserve)
        print('orig gem supply:',spl_env.gems)
        #print('card buying power:',self.)
        '''I'm using gem_reserve instead of buying power, I think it will be better to keep the values from gem reserve and cards separated to know what gems to resupply and which not'''
        for i in range(len(card.cost)):
            #update to know if we have to spend jokers
            over = max(0, new_cost[i]- self.gem_reserve[i]) #jokers needed for this color
            #print('index:',i,'over:',over)
            #update the number of reamining gems
            #print('Buying power:',self.buying_power[i],self.gem_reserve[i])
            gem_transaction = max(0, new_cost[i] - over)# after substracting cards + joker we get the amount to pay.
            #gem_transaction = max(0, card.cost[i] - self.buying_power[i] + self.gem_reserve[i])
            if over>self.gem_reserve[5]:
                 print(over,self.gem_reserve)
                 raise ValueError("Too many jokers")
            
            
            #print('gem_transaction?',gem_transaction)
            self.gem_reserve[i] -= gem_transaction
            spl_env.gems[i]  += gem_transaction
            self.gem_reserve[5]-=over
            spl_env.gems[5]+=over
        print('New gem reserve:',self.gem_reserve)
        print('new gem supply:',spl_env.gems)


        ##update number of jokers    
        #self.gem_reserve[5] -= over
        #spl_env.gems[5] += over


    def reserve_card(self, card, spl_env):
        '''We reserve the selected card and remove it from the board'''

        #add card to players reserved cards
        self.player_reserved_cards.append(card)

        #get rid of the card on the board
        if card.level ==1:
             spl_env.board_1.remove(card)
        
        elif card.level ==2:
             spl_env.board_2.remove(card)

        elif card.level ==3:
             spl_env.board_3.remove(card)

        #get a joker
        self.gem_reserve[5] += 1 # add joker to hand.
        spl_env.gems[5]-=1 # added removal of joker from board

    def buy_reserved_card(self, card, spl_env):
        '''We buy the selected reserved card and update the environment'''
        cards_color = [x.color for x in self.player_cards]

        #get our current color portofolio = buying power without gems
        cards_sum = [0,0,0,0,0]
        for x in cards_color:
             cards_sum[x] +=1
        
        new_cost=[0,0,0,0,0]
        for i in range(len(card.cost)):
            new_cost[i]=max(0,card.cost[i]-cards_sum[i])
        
        print('Discounted card cost:',new_cost)
        # update gem and card supply and player's state when a card is bought
        #add card to players cards
        self.player_cards.append(card)

        #compute required tokens
        over = 0
        print('reserve',self.gem_reserve)
        for i in range(len(card.cost)):
            #update to know if we have to spend jokers
            #over += max(0, card.cost[i]- self.buying_power[i])
            over=max(0,new_cost[i]-self.gem_reserve[i])
            #update the number of reamining gems
            #gem_transaction = max(0, card.cost[i] - self.buying_power[i] + self.gem_reserve[i])
            if over>self.gem_reserve[5]:
                 print(over, self.gem_reserve)
                 raise ValueError("Too many jokers")
            gem_transaction = max(0,new_cost[i]-over)
            self.gem_reserve[i] -=  gem_transaction
            spl_env.gems[i]  += gem_transaction
            spl_env.gems[5]+=over
            self.gem_reserve[5]-=over
        #update number of jokers    
        #self.gem_reserve[5] -= over
        #spl_env.gems[5] += over


    def take_gems(self, gems_array, spl_env):
        '''Add gems from supply to player's reserve and remove it from supply.'''

        # simply add the gems to the players count
        for i in range(len(gems_array)):
            self.gem_reserve[i] += gems_array[i]
            spl_env.gems[i] -= gems_array[i]
    
    def verify_noble(self, spl_env):
        '''Verify if we can get a noble'''

        #extract the colors of the cards
        cards_color = [x[1] for x in self.player_cards]

        #get our current color portfolio
        cards_sum = [0,0,0,0,0]
        for x in cards_color:
             cards_sum[x] +=1
        
        #check for the nobles
        for noble in spl_env.nobles:
             if sum([min(cards_sum[i]- noble[0][i], 0) for i in range(len(cards_sum))]) >=0:
                  self.nobles.append(noble)
                  spl_env.nobles.remove(noble)
    

    def count_point(self):
        '''Count points and win condition'''

        points =  0
        win = False
        for cards in self.player_cards:
              points +=cards.points
        
        for noble in self.nobles:
             points +=noble[1]
        
        if points >= 15:
             win = True
        self.points = points
        return points, win

    

In [6]:
class SplendorEnvironment:
    '''Set up the environment: Decks, Boards, Nobles, Gem supply, update and reset function'''

    def __init__(self, card_supply, deck_nobles, initial_gems = [7,7,7,7,7,5]):
    
        self.agent = SplendorAgent()
        #creates the card decks
        deck=cp.deepcopy(card_supply) #since we are using pop we need to define a new list removing references. 
        self.deck_1 = deck[0]
        self.deck_2 = deck[1]
        self.deck_3 = deck[2]

        #creates the cardboard
        self.board_1 = []
        self.board_2 = []
        self.board_3 = []

        self.UpdateCardBoard()

        #creates the noble board
        self.nobles = []
        self.CreateNobles(deck_nobles)

        #Create the gems
        self.gems= initial_gems
        
    def check_gems(self): # in theory if our code is right we wouldn't need this.
        for i in range(len(self.gems)):
            if i==5 and self.gems[i]>5:
                self.gems[i]=5
            elif self.gems[i]>7:
                self.gems[i]=7

    def UpdateCardBoard(self):
        '''Randomly shuffle each deck and take the first 3 cards to place them on the board
        Updates boards to 3 if there are less than 3 cards'''
        random.shuffle(self.deck_1)
        random.shuffle(self.deck_2)
        random.shuffle(self.deck_3)
        #update cards rank 1
        while len(self.board_1) <4:
            if len(self.deck_1) ==0:
                break
            self.board_1.append(self.deck_1.pop(0))
            
        #update cards rank 2
        while len(self.board_2) <4:
            if len(self.deck_2) ==0:
                break
            self.board_2.append(self.deck_2.pop(0))
            
        #update cards rank 3
        while len(self.board_3) <4:
            if len(self.deck_3) ==0:
                break
            self.board_3.append(self.deck_3.pop(0))
    
    
    def CreateNobles(self, deck_nobles):
        '''Shuffle the deck of Nobles and get the first 3 cards - no replacement here.'''
        deck_n=cp.deepcopy(deck_nobles)
        random.shuffle(deck_n)

        #update cards rank 1
        while len(self.nobles) <4:
            self.nobles.append(deck_n.pop(0)) #the issue for reseting is that when we pop we modify the original
           
      

    #Make this work
    def reset(self, new_deck,new_deck_nobles, initial_gems = [7,7,7,7,7,5]):
        '''Reset function, re-initializes all environmental variables:
        Set score to 0, reset supply, decks, boards, reserved cards, nobles etc.'''
        ###Reset all variables and rebuild:
        ##Decks:
        self.deck_1=[]
        self.deck_2=[]
        self.deck_3=[]

        #creates the card decks
         
        #self.deck_1 = new_deck[0]
        #self.deck_2 = new_deck[1]
        #self.deck_3 = new_deck[2]
        self.deck_1 = cp.deepcopy(new_deck[0])
        self.deck_2 = cp.deepcopy(new_deck[1])
        self.deck_3 = cp.deepcopy(new_deck[2])
        ##Boards:
        #creates the cardboard
        self.board_1 = []
        self.board_2 = []
        self.board_3 = []

        self.UpdateCardBoard()
        ##Nobles:
        #creates the noble board
        self.nobles = []
        self.CreateNobles(new_deck_nobles)
        ##Gems:
        #Create the gems
        self.gems= [7,7,7,7,7,5]
        ##Agentwise:
        # Reset the agent's variables
        self.agent.reset()
        
       
    '''This functions were here before, we keep tham as reference in case we want to take inspiration but we are not using them:
    '''
    def draw_card(self, player_id):
        # draw a card from the top of the deck and add it to the player's hand
        level = min(sum([len(x) for x in self.state.card_supply]), 4) - 1
        if len(self.state.card_supply[level]) == 0:
            self.state.card_supply[level] = self.state.card_supply[level + 1][::-1]
            self.state.card_supply[level + 1] = []
        card = self.state.card_supply[level].pop()
        self.state.players_state[player_id]['cards'][card.color] += 1

    def step(self, action):
        # execute the specified action and update the current player
        self.state.execute_action(action)
        self.current_player = (self.current_player + 1) % self.num_players

    def get_legal_actions(self):
        return self.state.get_legal_actions()

    def get_current_player(self):
        return self.current_player

    def get_state(self):
        return self.state

    def is_terminal(self):
        # check if the game is over (i.e., if any player has 15 prestige points or if the card and noble supplies are exhausted)
        if any([player['prestige_points'] >= 15 for player in self.state.players_state]):
            return True
        if sum([len(x) for x in self.state.card_supply]) == 0 and len(self.state.nobles) == 0:
            return True
        return False

    def get_reward(self):
        # calculate the reward for each player based on their score
        rewards = [0] * self.num_players
        max_score = max([self.state.get_player_score(i) for i in range(self.num_players)])
        for i in range(self.num_players):
            if self.state.get_player_score(i) == max_score:
                rewards[i] = 1
        return rewards

In [7]:
#mapping  {0: 'black', 1:'blue', 2: 'green', 3: 'red', 4:'white'}

# create cards rank 1
cards_1= [Card(1, 2, [0,1,1,1,1], 0),
          Card(1, 2, [1,1,1,1,0], 0),
          Card(1, 2, [0,2,0,2,1], 0),
          Card(1, 3, [0,0,0,4,0], 0),
          Card(1, 1, [0,0,4,0,0], 0),
          Card(1, 0, [1,2,1,0,1], 0)]

#create cards rank 2
cards_2=[Card(2, 2, [0,0,1,2,1], 1),
         Card(2, 2, [2,0,3,0,0], 1),
         Card(2, 2, [0,0,4,0,2], 1),
         Card(2, 3, [0,0,0,0,5], 1),
         Card(2, 1, [3,0,0,3,0], 1),
         Card(2, 0, [0,0,0,2,2], 1)]

#create cards rank3
cards_3=[Card(3, 2, [0,0,3,3,3], 3),
         Card(3, 2, [0,0,0,6,0], 3),
         Card(3, 2, [0,3,3,0,3], 3),
         Card(3, 3, [3,3,3,0,0], 4),
         Card(3, 1, [0,0,7,0,0], 4),
         Card(3, 0, [3,3,0,3,0], 4)]

# create nobles
nobles= [Noble([3,3,0,0,3], 3),
Noble([3,0,3,3,0], 3),
Noble([0,4,0,4,0], 3),
Noble([0,0,4,0,4], 3),
Noble([4,0,0,0,4], 3),
Noble([0,0,0,5,3], 3)]

# create SplendorState
'''Note, card supply and nobles are list of lists of lengths 3 and 1 respectively.'''
##Get the card supply:
card_supply = [cards_1, cards_2, cards_3]
###Initialize the environment with card_supply and nobles
env = SplendorEnvironment(card_supply, nobles)
##Initialize Agent:
'''Fernando is a random action agent, he is chill'''
Fernando = SplendorAgent()

### Gym env:

In [8]:
class SplendorEnv(gym.Env):
    def __init__(self, card_supply, deck_nobles, initial_gems=[7, 7, 7, 7, 7, 5]):
        super(SplendorEnv, self).__init__()

        self.env = SplendorEnvironment(card_supply, deck_nobles, initial_gems)

        # Define action and observation spaces
        self.action_space = ...
        self.observation_space = ...

    def step(self, action):
        # Execute the given action
        self.env.agent.execute_action(action, self.env)
        self.env.agent.UpdateBuyingPower()

        # Update the environment
        self.env.UpdateCardBoard()

        # Calculate reward and check if the game is done
        points, _ = self.env.agent.count_point()
        done = points >= 15
        reward = -1 # I'm setting reward to-1  for every move, alternatively could be number of points or smth else.

        # Get the current state (observation)
        observation = ...

        return observation, reward, done, {}

    def reset(self):
        # Reset the environment and agent
        self.env.reset(card_supply, nobles)
        self.env.agent.reset()

        # Get the initial state (observation)
        observation = ...

        return observation


action space: check these numbers and think more:
Taking gems: There are 5 colored gems (not counting jokers) and an agent can take up to 2 of the same color, or take 3 different colors. So there are (5 choose 2) + (5 choose 3) = 10 + 10 = 20 possibilities for taking gems.

Reserving cards: There are 12 cards on the board and 4 possible cards to reserve from each deck. So there are 12 + 4 * 3 = 24 possible reserve actions.

Purchasing cards: An agent can purchase any of the 9 cards on the board. So there are 9 possible purchase actions. + buying reserved cards (3)

In total, there are 20 + 24 + 9 + 3 = 56 possible action

In [12]:
self.action_space = spaces.Discrete(56)


NameError: name 'self' is not defined

Mapping actions:

In [11]:
##Need to complete this:
# Mapping action index to action
self.action_mapping = {
    0: ('take_gems', (0, 1)),
    1: ('take_gems', (0, 2)),
    ...
    20: ('reserve_card', 0),
    21: ('reserve_card', 1),
    ...
    38: ('purchase_card', 0),
    39: ('purchase_card', 1),
    ...
}

# Mapping action to action index
self.inverse_action_mapping = {v: k for k, v in self.action_mapping.items()}

SyntaxError: ':' expected after dictionary key (3303479267.py, line 6)

Observation space:

In [ ]:
# Observations:
# - 4 cards for each of the 3 boards (12 cards in total, 6 attributes each: cost, points, gem type)
# - 3 nobles (5 attributes each: cost, points)
# - Gem supply (6 types, including jokers)
# - Agent state:
#   - Gems (6 types, including jokers)
#   - Reserved cards (3 cards max, 6 attributes each: cost, points, gem type)
#   - Purchased cards (same format as the board cards)
#   - Points

card_space = spaces.Box(low=0, high=10, shape=(6,), dtype=int)
noble_space = spaces.Box(low=0, high=10, shape=(5,), dtype=int)
gem_space = spaces.Box(low=0, high=7, shape=(6,), dtype=int)

self.observation_space = spaces.Dict({
    'board_1': spaces.Tuple([card_space] * 4),
    'board_2': spaces.Tuple([card_space] * 4),
    'board_3': spaces.Tuple([card_space] * 4),
    'nobles': spaces.Tuple([noble_space] * 3),
    'gem_supply': gem_space,
    'agent_state': spaces.Dict({
        'gems': gem_space,
        'reserved_cards': spaces.Tuple([card_space] * 3),
        'purchased_cards': spaces


### Playing:

In [9]:
env = SplendorEnv(card_supply, nobles)

observation = env.reset()
done = False
actions_list = []

while not done:
    actions = env.env.agent.get_legal_actions(env.env)
    
    if len(actions) == 0:
        raise ValueError("No actions available?? check why")

    random.shuffle(actions)
    action = actions[0]

    observation, reward, done, info = env.step(action)
    actions_list.append(action)

print(f'Fernando made {reward} points in {len(actions_list)} turns, by making the following actions {actions_list}')


Gem_supply: [7, 7, 7, 7, 7, 5]
Gem_supply: [7, 5, 7, 7, 7, 5]
Gem_supply: [5, 5, 7, 7, 7, 5]
Gem_supply: [5, 5, 7, 7, 7, 4]
Gem_supply: [3, 5, 7, 7, 7, 4]
Gem_supply: [3, 4, 7, 6, 6, 4]
Gem_supply: [3, 4, 7, 6, 6, 3]
Gem_supply: [3, 4, 7, 6, 6, 2]
cards power: [0, 0, 0, 0, 0]
cost: [2, 0, 3, 0, 0]
Discounted card cost: [2, 0, 3, 0, 0]
orig gem reserve: [4, 3, 0, 1, 1, 3]
orig gem supply: [3, 4, 7, 6, 6, 2]
New gem reserve: [2, 3, 0, 1, 1, 0]
new gem supply: [5, 4, 7, 6, 6, 5]
Gem_supply: [5, 4, 7, 6, 6, 5]
Gem_supply: [5, 4, 5, 6, 6, 5]
Gem_supply: [5, 4, 3, 6, 6, 5]
cards power: [0, 0, 1, 0, 0]
cost: [0, 0, 4, 0, 0]
Discounted card cost: [0, 0, 3, 0, 0]
orig gem reserve: [2, 3, 4, 1, 1, 0]
orig gem supply: [5, 4, 3, 6, 6, 5]
New gem reserve: [2, 3, 1, 1, 1, 0]
new gem supply: [5, 4, 6, 6, 6, 5]
Gem_supply: [5, 4, 6, 6, 6, 5]
Gem_supply: [4, 4, 5, 6, 6, 5]
cards power: [0, 1, 1, 0, 0]
cost: [3, 3, 3, 0, 0]
Discounted card cost: [3, 2, 2, 0, 0]
orig gem reserve: [3, 3, 2, 1, 1, 0]
orig 